# Assymetric organocatalysis

Asymmetric organocatalysis allows the synthesis of enantiopure compounds using organic molecules as catalysts. In 2021 B. List and D. MacMillan were awarded the Nobel Prize for the development of asymmetric organocatalysis. Recently, Denmark and coworkers published a reaction of asymmetric addition of thiols to imines catalyzed by phosphoric acids (Figure 1).

<img src="img/figure_1.png" width="450"/>

**Figure 1. Asymmetric addition of thiol to imines catalyzed by chiral phosphoric acid catalysts**

# 3D Catalyst descriptors

Each reaction was transformed to a Condensed Graph of Reaction (CGR) with a CGRtools package. Given CGRs were encoded by ISIDA (In Silico Design and Data Analysis) fragment descriptors.

<img src="img/figure_2.png" width="500"/>

**Figure 2. Addition of thiols to imine and related Condensed Graph of Reaction (CGR).**

For each catalyst, up to 50 conformations (**nconfs**) within a 50 kcal/mol energy window (**energy**) were generated using the distance geometry algorithm implemented in RDKit.  The generated catalyst conformations were encoded with pmapper 3D descriptors. The descriptor value is defined as the number of occurrences of particular 3D atom triplet in the catalyst conformation. The atom triplet was defined by (1) the type of atoms (C, N, O, S, P, F, Cl, Br, I, 5-membered and 6-membered aromatic ring) and (2) the distance between atoms in triplet.

<img src="img/figure_3.png" width="600"/>

**Figure 3. Workflow of preparation of pmapper descriptors for a given conformation. (1) given 2D catalyst structure; (2) generation of 3D catalyst conformation and assigning atom labels; (3) generation of a 3D fully connected graph for which an ensemble of triplets is generated (for demonstration, graph of four atoms is chosen); (4) enumeration of all atom triplets; (5) counting of atom triplets in all conformations.**

Vectors of 2D fragment reaction descriptors and 3D physicochemical quadruplets were then concatenated to form combined reaction/catalyst descriptor vector. 
<img src="img/figure_4.png" width="800"/>

**Figure 4. Preparation of descriptors encoding reaction/catalyst combinations. A chemical reaction is encoded by m CGR/ISIDA descriptors. A catalyst is represented by its N conformations, each encoded by n of 3D atom triplet descriptors. Concatenation of reaction and catalyst descriptors results in the vector of (m + n) size.**

# Multi-Instance learning

Multi-instance (MI) machine learning approaches can be used to solve the issues of representation of each molecule by multiple conformations (instances) and automatic selection of the most relevant ones. In the multi-instance approach, an example (i.e., a molecule) is presented by a bag of instances (i.e., a set of conformations), and a label (a molecule property value) is available only for a bag (a molecule), but not for individual instances (conformations).

Here, we report an application of Multi-Instance Learning approach to predictive modeling of enantioselectivity of chiral catalysts. Catalysts were represented by ensembles of conformations encoded by the *pmapper* 3D descriptors.

<img src="img/figure_5.png" width="550"/>

**Figure 5. Mutli-instance learning approach**


# Instance-Wrapper algorithm

Instance-Wrapper is the simplest instance-based MI algorithm, where each training instance (conformation) is assigned the same experimental enantioselectivity (Figure 6). As a result, we obtain a data set where each conformation is an individual training object and any conventional machine learning algorithm can be applied to build the model. Given a new catalyst, the enantioselectivity is predicted for each conformation, and then conformation predictions are averaged to derive the final predicted enantioselectivity of the catalyst (Figure 6). 

<img src="img/figure_6.png" width="900"/>

**Figure 6. Instance-Wrapper algorithm**

# 0. Code installation

Using **conda** and **pip** is the easiest way to install all required packages. Create a new environment (named "exp") with Python 3.6. Note the issues related to RDKit installation https://stackoverflow.com/questions/70202430/rdkit-importerror-dll-load-failed. <br/>

Run these commands in the command line:

`conda create -n exp python=3.6`<br/>
`conda activate exp` <br/>

Install RDKit package: <br/>

`conda install -c conda-forge rdkit` <br/>

Install PyTorch package: <br/>

`conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch` <br/>
`pip install torch_optimizer` <br/>

Install software to calculate 2D reactant descriptors: <br/>

`pip install CGRTools CIMTools` <br/>

Install software to calculate 3D catalyst descriptors: <br/>

`conda install -c conda-forge openbabel` <br/>
` pip install networkx` <br/>
`pip install pmapper` <br/>

# 1. Data set preparation

The reactant transformations and catalyst structures are recorded in an RDF file, which has a fixed structure: 

**Reactants mol blocks** - encoded reactants transformation <br/>

Metadata fields `$DTYPE-$DATUM`:<br/>
**ID** - catalyst ID <br/>
**SELECTIVITY** - experimental catalyst selectivity <br/>
**CATALYST_SMILES** - сatalyst structure in SMILES <br/>

You should prepare your data set in the described form, also see example RDF file in `data/input_data.rdf`

# 2. Descriptor calculation


In [ ]:
import os
from miqssr.utils import calc_3d_descr

input_fname = os.path.join('data', 'input_data.rdf') # data
nconfs = [5] # max number of conformers to generate
energy = 50 # energy window
ncpu = 20 # number of cpus
path = './descriptors' # where to store the calculated descriptors

out_fname = calc_3d_descr(input_fname=input_fname, nconfs=nconfs, energy=energy, ncpu=ncpu, path=path)

The descriptor folder contains several files:

`2DDescrISIDA_cgr-data_0.csv` - fragment descriptors of reactant transformations<br/>

`conf-catalyst_data_5.pkl` - pickle file with RDKit the generated conformations<br/>
`conf-50_catalyst_data_log.pkl` - pickle file with the conformation energies<br/>

`PhFprPmapper_conf-catalyst_data_5.txt` - SVM file with pmapper 3D descriptors for generated conformations<br/>
`PhFprPmapper_conf-catalyst_data_5.colnames` - names of pmapper 3D descriptors for generated conformations<br/>
`PhFprPmapper_conf-catalyst_data_5.rownames` - id of generated conformations<br/>

`PhFprPmapper_concat_data_5.txt` - SVM file with concatenated reactant 2D fragment descriptors and pmapper 3D descriptors<br/>
`PhFprPmapper_concat_data_5.rownames` - reaction id and experimental enantioselectivity<br/>

# 3. Preparation of training and test set

Reaction descriptors are stored in a dense SVM format. One should to implement a function to create a n × m × k list of bags (n - number of reactions, m - bag size (number of conformations generated), k - number of descriptors).

Training set containes 384 reactions (24 catalysts × 16 reactant combinations = 384 reactions), and the external test set containes 691 reactions.

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd

def load_svm_data(fname):
    
    def str_to_vec(dsc_str, dsc_num):

        tmp = {}
        for i in dsc_str.split(' '):
            tmp[int(i.split(':')[0])] = int(i.split(':')[1])
        #
        tmp_sorted = {}
        for i in range(dsc_num):
            tmp_sorted[i] = tmp.get(i, 0)
        vec = list(tmp_sorted.values())

        return vec
    
    #
    with open(fname) as f:
        dsc_tmp = [i.strip() for i in f.readlines()]

    with open(fname.replace('txt', 'rownames')) as f:
        react_names = [i.strip() for i in f.readlines()]
    #
    labels_tmp = [float(i.split(':')[1]) for i in react_names]
    idx_tmp = [i.split(':')[0] for i in react_names]
    dsc_num = max([max([int(j.split(':')[0]) for j in i.strip().split(' ')]) for i in dsc_tmp])
    #
    bags, labels, idx = [], [], []
    for react_idx in list(np.unique(idx_tmp)):
        bag, labels_, idx_ = [], [], []
        for dsc_str, label, i in zip(dsc_tmp, labels_tmp, idx_tmp):
            if i == react_idx:
                bag.append(str_to_vec(dsc_str, dsc_num))
                labels_.append(label)
                idx_.append(i)
                
        bags.append(np.array(bag).astype('uint8'))
        labels.append(labels_[0])
        idx.append(idx_[0])

    return np.array(bags), np.array(labels), np.array(idx)


def train_test_split_function(bags, labels, idx, test_set='default'):

    with open(os.path.join('data', 'test_sets.pickle'), 'rb') as f:
        test_reactions_idx = pickle.load(f)[test_set]

    x_train, x_test = [], []
    y_train, y_test = [], []
    idx_train, idx_test = [], []
    for bag, label, i in zip(bags, labels, idx):
        if i in test_reactions_idx:
            x_test.append(bag)
            y_test.append(label)
            idx_test.append(i)
        else:
            x_train.append(bag)
            y_train.append(label)
            idx_train.append(i)
            
    x_train, x_test, y_train, y_test = np.array(x_train), np.array(x_test), np.array(y_train), np.array(y_test)

    return x_train, x_test, y_train, y_test, idx_train, idx_test


def train_test_split_default(bags, labels, idx):
    return train_test_split_function(bags, labels, idx, test_set='default')


# split data into a training and test set
dsc_fname = os.path.join('descriptors', 'PhFprPmapper_concat_data_5.txt') # descriptors file
bags, labels, idx = load_svm_data(dsc_fname)
print(f'There are {len(bags)} reactions encoded with {bags[0].shape[1]} descriptors')
x_train, x_test, y_train, y_test, idx_train, idx_test = train_test_split_default(bags, labels, idx)
print(f'There are {len(x_train)} training reactions and {len(x_test)} test reactions')

For better training of the neural network, the descriptors should be scaled:

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def scale_data(x_train, x_test):
    scaler = MinMaxScaler()
    scaler.fit(np.vstack(x_train))
    x_train_scaled = x_train.copy()
    x_test_scaled = x_test.copy()
    for i, bag in enumerate(x_train):
        x_train_scaled[i] = scaler.transform(bag)
    for i, bag in enumerate(x_test):
        x_test_scaled[i] = scaler.transform(bag)
    return np.array(x_train_scaled), np.array(x_test_scaled)


x_train_scaled, x_test_scaled = scale_data(x_train, x_test)

One should implement a protocol for optimizing the hyperparameters of the neural network. Here we assign the optimal hyperparameters found with the grid search technique.

In [ ]:
from miqssr.estimators.wrappers import InstanceWrapperMLPRegressor
from miqssr.estimators.utils import set_seed
set_seed(43)

ndim = (x_train_scaled[0].shape[1], 256, 128, 64) # number of hidden layers and neurons in the main network
pool = 'mean'                                       # type of pulling of instance descriptors
n_epoch = 1000                                       # maximum number of learning epochs
lr = 0.001                                          # learning rate
weight_decay = 0.001                                # l2 regularization
batch_size = 99999999                               # batch size
init_cuda = True                                    # True if GPU is available


net = InstanceWrapperMLPRegressor(ndim=ndim, pool=pool, init_cuda=init_cuda)
net.fit(x_train_scaled, y_train, 
        n_epoch=n_epoch, 
        lr=lr,
        weight_decay=weight_decay,
        batch_size=batch_size)

The trained model is then used to predict enantioselectivity on 3 test sets (see details in original paper):

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error

results_df = pd.DataFrame()
results_df['IDX'] = idx_test
results_df['OBS'] = y_test
results_df['PRED'] = net.predict(x_test_scaled)
results_df.set_index(['IDX'], inplace=True)

# results
with open(os.path.join('data', 'test_sets.pickle'), 'rb') as f:
    test_sets = pickle.load(f)

print('Test scenario 1: 9 test reactions × 24 training catalysts = 216 test data points')
tmp = results_df.loc[test_sets['test_1']]
r2 = r2_score(tmp['OBS'], tmp['PRED'])
mae = mean_absolute_error(tmp['OBS'], tmp['PRED'])
print(f'    R2 = {r2:.2f}')
print(f'    MAE = {mae:.2f} kcal/mol')

print('Test scenario 2: 9 training reactions × 19 test catalysts = 304 test data points')
tmp = results_df.loc[test_sets['test_2']]
r2 = r2_score(tmp['OBS'], tmp['PRED'])
mae = mean_absolute_error(tmp['OBS'], tmp['PRED'])
print(f'    R2 = {r2:.2f}')
print(f'    MAE = {mae:.2f} kcal/mol')

print('Test scenario 3: 9 test reactions × 19 test catalysts = 171 data points')
tmp = results_df.loc[test_sets['test_3']]
r2 = r2_score(tmp['OBS'], tmp['PRED'])
mae = mean_absolute_error(tmp['OBS'], tmp['PRED'])
print(f'    R2 = {r2:.2f}')
print(f'    MAE = {mae:.2f} kcal/mol')